# DeepFabric ile OpenRouter (GPT-OSS-20B) Kullanarak Sentetik Veri Üretimi

Bu notebook, Google Colab'da DeepFabric kütüphanesini kullanarak OpenRouter API üzerinden GPT-OSS-20B modeli ile sentetik veri üretimini göstermektedir.

## İçindekiler
1. Kurulum ve Konfigürasyon
2. OpenRouter API Bağlantısı
3. Temel Sentetik Veri Üretimi
4. Farklı Veri Türleri:
   - Soru-Cevap Çiftleri
   - Konuşma Verileri
   - Kod Örnekleri
   - Tool Calling Örnekleri
5. Veri Analizi ve Görselleştirme
6. Fine-tuning için Hazırlık

---

## 1. Kurulum ve Konfigürasyon

### DeepFabric Nedir?

DeepFabric, yapay zeka modelleri için yüksek kaliteli eğitim verileri oluşturmak üzere tasarlanmış bir sentetik veri üretim framework'üdür.

### OpenRouter Nedir?

OpenRouter, birden fazla LLM modelini tek bir API üzerinden kullanmanızı sağlayan bir servis. OpenAI-uyumlu API formatı kullanır.

### GPT-OSS-20B

20 milyar parametre içeren açık kaynak bir dil modeli.

In [ ]:
# DeepFabric ve gerekli kütüphaneleri kur
!pip install deepfabric datasets pandas matplotlib seaborn pyyaml -q

print("✅ Kurulum tamamlandı!")

In [ ]:
# Gerekli kütüphaneleri import et
import os
import json
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datasets import load_dataset
from google.colab import userdata

# Görselleştirme ayarları
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ Kütüphaneler yüklendi!")

## 2. OpenRouter API Konfigürasyonu

### OpenRouter API Key Nasıl Alınır?

1. https://openrouter.ai/ adresine gidin
2. Hesap oluşturun veya giriş yapın
3. API Keys bölümünden yeni bir key oluşturun
4. Key'i kopyalayın

### Google Colab'da API Key Kullanımı

**Yöntem 1: Colab Secrets (Önerilen)**
- Sol tarafta 🔑 (Secrets) simgesine tıklayın
- "Add new secret" ile `OPENROUTER_API_KEY` ekleyin

**Yöntem 2: Doğrudan Kod İçinde (Dikkatli kullanın)**
- Aşağıdaki hücrede `YOUR_OPENROUTER_API_KEY_HERE` yerine key'inizi yapıştırın

In [ ]:
# OpenRouter API Konfigürasyonu

# Yöntem 1: Colab Secrets kullanarak (Önerilen)
try:
    OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
    print("✅ API Key Colab Secrets'dan yüklendi")
except:
    # Yöntem 2: Manuel olarak girin (Sadece test için)
    OPENROUTER_API_KEY = "YOUR_OPENROUTER_API_KEY_HERE"  # Buraya kendi key'inizi girin
    print("⚠️  API Key manuel olarak girildi")

# OpenRouter konfigürasyonu
os.environ['OPENAI_API_KEY'] = OPENROUTER_API_KEY
os.environ['OPENAI_BASE_URL'] = 'https://openrouter.ai/api/v1'

# Model seçimi - OpenRouter'daki model adları
# GPT-OSS-20B veya başka modeller:
# - "mistralai/mistral-7b-instruct"
# - "meta-llama/llama-3.1-8b-instruct"
# - "qwen/qwen-2.5-7b-instruct"
# - "google/gemma-2-9b-it"

MODEL_NAME = "openai/gpt-3.5-turbo"  # GPT-OSS-20B için model adını buraya girin
PROVIDER = "openai"  # OpenRouter, OpenAI-uyumlu API kullanır

print(f"\n📡 OpenRouter Konfigürasyonu:")
print(f"  Base URL: {os.environ['OPENAI_BASE_URL']}")
print(f"  Provider: {PROVIDER}")
print(f"  Model: {MODEL_NAME}")
print(f"  API Key: {'*' * 20}{OPENROUTER_API_KEY[-8:]}" if len(OPENROUTER_API_KEY) > 8 else "  API Key: [SET]")

In [ ]:
# API bağlantısını test et (Opsiyonel)
import requests

headers = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json"
}

test_data = {
    "model": MODEL_NAME,
    "messages": [{"role": "user", "content": "Merhaba, bu bir test mesajıdır."}],
    "max_tokens": 50
}

try:
    response = requests.post(
        "https://openrouter.ai/api/v1/chat/completions",
        headers=headers,
        json=test_data,
        timeout=30
    )
    
    if response.status_code == 200:
        result = response.json()
        print("✅ OpenRouter API bağlantısı başarılı!")
        print(f"\nTest Yanıtı: {result['choices'][0]['message']['content'][:100]}...")
    else:
        print(f"❌ Bağlantı hatası: {response.status_code}")
        print(f"Hata mesajı: {response.text}")
except Exception as e:
    print(f"❌ Hata: {e}")

## 3. Temel Sentetik Veri Üretimi

DeepFabric'in CLI kullanarak basit bir sentetik veri seti oluşturalım.

In [ ]:
# Çalışma dizinleri oluştur
output_dir = Path('sentetik_veriler')
config_dir = Path('konfigurasyonlar')

output_dir.mkdir(exist_ok=True)
config_dir.mkdir(exist_ok=True)

print("✅ Dizinler oluşturuldu")
print(f"  Veri çıktı dizini: {output_dir}")
print(f"  Konfigürasyon dizini: {config_dir}")

In [ ]:
# Basit CLI kullanımı - İlk sentetik veri setimiz
!deepfabric generate \
  --mode tree \
  --provider {PROVIDER} \
  --model "{MODEL_NAME}" \
  --depth 2 \
  --degree 2 \
  --num-steps 8 \
  --topic-prompt "Yapay Zeka ve Makine Öğrenmesi Temelleri" \
  --dataset-save-as sentetik_veriler/temel_dataset.jsonl

print("\n✅ Temel veri seti oluşturuldu!")

In [ ]:
# Oluşturulan veriyi inceleyelim
def veri_goster(dosya_yolu, adet=2):
    """JSONL dosyasındaki verileri gösterir"""
    if not Path(dosya_yolu).exists():
        print(f"❌ Dosya bulunamadı: {dosya_yolu}")
        return
    
    print(f"\n📊 Veri Seti: {dosya_yolu}")
    print("=" * 80)
    
    with open(dosya_yolu, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= adet:
                break
            
            data = json.loads(line)
            print(f"\n🔹 Örnek {i+1}:")
            print("-" * 80)
            
            if 'messages' in data:
                for msg in data['messages']:
                    role = msg.get('role', 'unknown').upper()
                    content = msg.get('content', '')
                    print(f"\n[{role}]")
                    print(content[:300] if len(content) > 300 else content)
                    if len(content) > 300:
                        print("...")
            else:
                print(json.dumps(data, indent=2, ensure_ascii=False)[:500])
            
            print("\n" + "=" * 80)

veri_goster('sentetik_veriler/temel_dataset.jsonl', adet=2)

## 4. Farklı Türde Sentetik Veri Üretimi

Şimdi farklı use case'ler için özelleştirilmiş veri setleri oluşturalım.

### 4.1 Soru-Cevap Veri Seti (Q&A Dataset)

Eğitim ve bilgi amaçlı soru-cevap çiftleri üretelim.

In [ ]:
# Soru-Cevap konfigürasyonu
qa_config = f"""
dataset_system_prompt: |
  Sen yardımcı bir Türkçe AI asistanısın. Öğrencilere konuları açık, 
  anlaşılır ve detaylı şekilde anlatıyorsun.

topic_tree:
  topic_prompt: |
    Python Programlama Dili:
    - Temel veri yapıları (listeler, dict, tuple, set)
    - Fonksiyonlar ve lambda ifadeleri
    - Dosya işlemleri
    - Hata yönetimi (try-except)
    - Modüller ve paketler
  
  provider: "{PROVIDER}"
  model: "{MODEL_NAME}"
  temperature: 0.7
  degree: 3
  depth: 2

data_engine:
  generation_system_prompt: |
    Verilen konular hakkında Türkçe soru-cevap çiftleri oluştur.
    Her cevap:
    - Açık ve anlaşılır olmalı
    - Örnekler içermeli
    - Orta seviye teknik detay içermeli
    - Pratik kullanım alanlarını göstermeli
  
  temperature: 0.6
  provider: "{PROVIDER}"
  model: "{MODEL_NAME}"

dataset:
  creation:
    num_steps: 15
    batch_size: 2
    sys_msg: true
  template: "builtin://chatml.py"
"""

qa_config_path = config_dir / 'qa_config.yaml'
with open(qa_config_path, 'w', encoding='utf-8') as f:
    f.write(qa_config)

print(f"✅ Soru-Cevap konfigürasyonu oluşturuldu: {qa_config_path}")

In [ ]:
# Soru-Cevap veri setini üret
!deepfabric generate \
  --config {qa_config_path} \
  --dataset-save-as sentetik_veriler/python_qa_dataset.jsonl

print("\n✅ Python Soru-Cevap veri seti oluşturuldu!")
veri_goster('sentetik_veriler/python_qa_dataset.jsonl', adet=2)

### 4.2 Konuşma Veri Seti (Conversation Dataset)

Çok turlu, gerçekçi konuşmalar için veri seti.

In [ ]:
# Konuşma veri seti konfigürasyonu
conversation_config = f"""
dataset_system_prompt: |
  Sen müşteri hizmetleri temsilcisisin. Müşterilere yardımcı, 
  nazik ve çözüm odaklı yaklaşıyorsun.

topic_tree:
  topic_prompt: |
    E-ticaret Müşteri Hizmetleri Senaryoları:
    - Sipariş takibi ve teslimat sorunları
    - Ürün iade ve değişim işlemleri
    - Ödeme ve fatura sorunları
    - Ürün bilgileri ve öneriler
    - Hesap ve üyelik işlemleri
  
  provider: "{PROVIDER}"
  model: "{MODEL_NAME}"
  temperature: 0.8
  degree: 3
  depth: 2

data_engine:
  generation_system_prompt: |
    Müşteri-temsilci konuşmaları oluştur. Her konuşma:
    - Gerçekçi ve doğal olmalı
    - 3-5 mesaj alışverişi içermeli
    - Müşterinin sorununu çözmeli
    - Profesyonel ama samimi olmalı
    - Türkçe yazım ve ifade kurallarına uymalı
  
  temperature: 0.7
  provider: "{PROVIDER}"
  model: "{MODEL_NAME}"

dataset:
  creation:
    num_steps: 12
    batch_size: 2
    sys_msg: true
  template: "builtin://chatml.py"
"""

conv_config_path = config_dir / 'conversation_config.yaml'
with open(conv_config_path, 'w', encoding='utf-8') as f:
    f.write(conversation_config)

print(f"✅ Konuşma konfigürasyonu oluşturuldu: {conv_config_path}")

In [ ]:
# Konuşma veri setini üret
!deepfabric generate \
  --config {conv_config_path} \
  --dataset-save-as sentetik_veriler/musteri_konusma_dataset.jsonl

print("\n✅ Müşteri konuşma veri seti oluşturuldu!")
veri_goster('sentetik_veriler/musteri_konusma_dataset.jsonl', adet=1)

### 4.3 Kod Örnekleri Veri Seti (Code Examples Dataset)

Programlama öğretimi için kod örnekleri içeren veri seti.

In [ ]:
# Kod örnekleri konfigürasyonu
code_config = f"""
dataset_system_prompt: |
  Sen bir programlama eğitmenisin. Her yanıtta çalışan, 
  açıklamalı kod örnekleri veriyorsun.

topic_tree:
  topic_prompt: |
    Python ile Veri Analizi:
    - Pandas ile veri manipülasyonu
    - NumPy ile matematiksel işlemler
    - Matplotlib ile görselleştirme
    - Veri temizleme ve preprocessing
    - İstatistiksel analizler
  
  provider: "{PROVIDER}"
  model: "{MODEL_NAME}"
  temperature: 0.6
  degree: 3
  depth: 2

data_engine:
  generation_system_prompt: |
    Veri analizi hakkında soru-cevap üret. Her cevap:
    1. Konsepti kısaca açıkla
    2. Çalışan Python kodu ver
    3. Kodu satır satır açıkla
    4. Beklenen çıktıyı göster
    5. Pratik kullanım örnekleri ver
  
  temperature: 0.5
  provider: "{PROVIDER}"
  model: "{MODEL_NAME}"

dataset:
  creation:
    num_steps: 12
    batch_size: 2
    sys_msg: true
  template: "builtin://chatml.py"
"""

code_config_path = config_dir / 'code_config.yaml'
with open(code_config_path, 'w', encoding='utf-8') as f:
    f.write(code_config)

print(f"✅ Kod örnekleri konfigürasyonu oluşturuldu: {code_config_path}")

In [ ]:
# Kod örnekleri veri setini üret
!deepfabric generate \
  --config {code_config_path} \
  --dataset-save-as sentetik_veriler/python_kod_ornekleri.jsonl

print("\n✅ Python kod örnekleri veri seti oluşturuldu!")
veri_goster('sentetik_veriler/python_kod_ornekleri.jsonl', adet=1)

### 4.4 Tool Calling Veri Seti

Fonksiyon çağırma ve API kullanımı için veri seti.

In [ ]:
# Tool calling konfigürasyonu
tool_config = f"""
dataset_system_prompt: |
  Sen bir API yardımcısısın. Kullanıcılara doğru fonksiyonları 
  seçip çağırmayı öğretiyorsun.

topic_tree:
  topic_prompt: |
    API ve Fonksiyon Çağırma Senaryoları:
    - Veritabanı sorguları (SQL, NoSQL)
    - Dosya işlemleri (okuma, yazma, silme)
    - HTTP istekleri (GET, POST, PUT, DELETE)
    - Veri dönüşümleri ve validasyon
    - Asenkron işlemler
  
  provider: "{PROVIDER}"
  model: "{MODEL_NAME}"
  temperature: 0.7
  degree: 3
  depth: 2

data_engine:
  generation_system_prompt: |
    Fonksiyon çağırma örnekleri oluştur. Her örnek:
    - Kullanıcı isteğini içermeli
    - Uygun fonksiyon(lar)ı seçmeli
    - Parametreleri doğru belirlenmeli
    - Fonksiyon çıktısını yorumlamalı
    - Hata durumlarını ele almalı
  
  temperature: 0.6
  provider: "{PROVIDER}"
  model: "{MODEL_NAME}"

dataset:
  creation:
    num_steps: 10
    batch_size: 2
    sys_msg: true
  template: "builtin://chatml.py"
"""

tool_config_path = config_dir / 'tool_config.yaml'
with open(tool_config_path, 'w', encoding='utf-8') as f:
    f.write(tool_config)

print(f"✅ Tool calling konfigürasyonu oluşturuldu: {tool_config_path}")

In [ ]:
# Tool calling veri setini üret
!deepfabric generate \
  --config {tool_config_path} \
  --dataset-save-as sentetik_veriler/tool_calling_dataset.jsonl

print("\n✅ Tool calling veri seti oluşturuldu!")
veri_goster('sentetik_veriler/tool_calling_dataset.jsonl', adet=1)

## 5. Veri Analizi ve Görselleştirme

Oluşturduğumuz tüm veri setlerini analiz edelim.

In [ ]:
def veri_seti_analizi(dosya_yolu):
    """JSONL veri setini analiz eder"""
    if not Path(dosya_yolu).exists():
        return None
    
    veriler = []
    with open(dosya_yolu, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                veriler.append(json.loads(line))
            except:
                pass
    
    if not veriler:
        return None
    
    # İstatistikler
    toplam = len(veriler)
    mesaj_sayilari = []
    mesaj_uzunluklari = []
    
    for veri in veriler:
        if 'messages' in veri:
            mesaj_sayilari.append(len(veri['messages']))
            for msg in veri['messages']:
                if 'content' in msg:
                    mesaj_uzunluklari.append(len(msg['content']))
    
    return {
        'dosya': Path(dosya_yolu).name,
        'toplam_ornek': toplam,
        'ort_mesaj_sayisi': sum(mesaj_sayilari) / len(mesaj_sayilari) if mesaj_sayilari else 0,
        'ort_mesaj_uzunlugu': sum(mesaj_uzunluklari) / len(mesaj_uzunluklari) if mesaj_uzunluklari else 0,
        'toplam_mesaj': len(mesaj_uzunluklari)
    }

# Tüm veri setlerini analiz et
veri_setleri = [
    'sentetik_veriler/temel_dataset.jsonl',
    'sentetik_veriler/python_qa_dataset.jsonl',
    'sentetik_veriler/musteri_konusma_dataset.jsonl',
    'sentetik_veriler/python_kod_ornekleri.jsonl',
    'sentetik_veriler/tool_calling_dataset.jsonl'
]

analizler = []
for dosya in veri_setleri:
    analiz = veri_seti_analizi(dosya)
    if analiz:
        analizler.append(analiz)

# DataFrame oluştur
df_analiz = pd.DataFrame(analizler)

print("\n📊 Veri Setleri Özet İstatistikleri:")
print("=" * 100)
print(df_analiz.to_string(index=False))
print("=" * 100)
print(f"\n✅ Toplam {df_analiz['toplam_ornek'].sum():.0f} örnek üretildi!")
print(f"✅ Toplam {df_analiz['toplam_mesaj'].sum():.0f} mesaj oluşturuldu!")

In [ ]:
# Görselleştirme
if len(analizler) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Grafik 1: Veri seti boyutları
    axes[0].bar(range(len(df_analiz)), df_analiz['toplam_ornek'], 
                color=['#3498db', '#e74c3c', '#2ecc71', '#f39c12', '#9b59b6'])
    axes[0].set_xticks(range(len(df_analiz)))
    axes[0].set_xticklabels([f.replace('_dataset.jsonl', '').replace('_', ' ').title() 
                              for f in df_analiz['dosya']], rotation=45, ha='right')
    axes[0].set_ylabel('Örnek Sayısı', fontsize=11)
    axes[0].set_title('Veri Seti Boyutları', fontsize=13, fontweight='bold')
    axes[0].grid(axis='y', alpha=0.3)
    
    # Grafik 2: Ortalama mesaj sayıları
    axes[1].bar(range(len(df_analiz)), df_analiz['ort_mesaj_sayisi'], 
                color=['#3498db', '#e74c3c', '#2ecc71', '#f39c12', '#9b59b6'])
    axes[1].set_xticks(range(len(df_analiz)))
    axes[1].set_xticklabels([f.replace('_dataset.jsonl', '').replace('_', ' ').title() 
                              for f in df_analiz['dosya']], rotation=45, ha='right')
    axes[1].set_ylabel('Ortalama Mesaj/Örnek', fontsize=11)
    axes[1].set_title('Örnek Başına Ortalama Mesaj Sayısı', fontsize=13, fontweight='bold')
    axes[1].grid(axis='y', alpha=0.3)
    
    # Grafik 3: Ortalama mesaj uzunlukları
    axes[2].bar(range(len(df_analiz)), df_analiz['ort_mesaj_uzunlugu'], 
                color=['#3498db', '#e74c3c', '#2ecc71', '#f39c12', '#9b59b6'])
    axes[2].set_xticks(range(len(df_analiz)))
    axes[2].set_xticklabels([f.replace('_dataset.jsonl', '').replace('_', ' ').title() 
                              for f in df_analiz['dosya']], rotation=45, ha='right')
    axes[2].set_ylabel('Karakter Sayısı', fontsize=11)
    axes[2].set_title('Ortalama Mesaj Uzunluğu', fontsize=13, fontweight='bold')
    axes[2].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('sentetik_veriler/analiz_grafikleri.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n✅ Grafikler kaydedildi: sentetik_veriler/analiz_grafikleri.png")
else:
    print("⚠️  Analiz için yeterli veri yok")

## 6. Veri Setlerini Birleştirme ve Fine-tuning için Hazırlama

In [ ]:
# Tüm veri setlerini birleştir
birlesik_veri = []

for dosya in veri_setleri:
    if Path(dosya).exists():
        with open(dosya, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    birlesik_veri.append(json.loads(line))
                except:
                    pass

# Birleştirilmiş veri setini kaydet
birlesik_dosya = output_dir / 'birlesik_dataset.jsonl'
with open(birlesik_dosya, 'w', encoding='utf-8') as f:
    for veri in birlesik_veri:
        f.write(json.dumps(veri, ensure_ascii=False) + '\n')

print(f"✅ Birleştirilmiş veri seti oluşturuldu: {birlesik_dosya}")
print(f"   Toplam örnek: {len(birlesik_veri)}")

In [ ]:
# HuggingFace datasets formatına çevir
try:
    dataset = load_dataset('json', data_files=str(birlesik_dosya))
    print("✅ Veri seti HuggingFace formatında yüklendi!")
    print(f"\nVeri seti bilgileri:")
    print(dataset)
    
    if len(dataset['train']) > 0:
        print("\n📝 Örnek bir satır:")
        print(json.dumps(dataset['train'][0], indent=2, ensure_ascii=False)[:500])
        print("...")
except Exception as e:
    print(f"⚠️  Hata: {e}")

## 7. Fine-tuning Örnek Kodu

Üretilen veri ile model fine-tuning yapmak için örnek kod.

In [ ]:
# Fine-tuning örnek kodu
finetuning_kod = '''
# Model Fine-tuning için Örnek Kod
# Gerekli kütüphaneler: transformers, trl, peft, accelerate, bitsandbytes

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import torch

# 1. Model ve tokenizer yükle
model_name = "meta-llama/Llama-3.2-1B-Instruct"  # veya başka bir model

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True  # QLoRA için
)

# 2. Veri setini yükle
dataset = load_dataset("json", data_files="sentetik_veriler/birlesik_dataset.jsonl")

# 3. LoRA konfigürasyonu
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]
)

# 4. Training parametreleri
training_args = TrainingArguments(
    output_dir="./finetuned_model",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    warmup_steps=50,
    optim="paged_adamw_8bit"
)

# 5. Trainer oluştur
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    tokenizer=tokenizer,
    peft_config=peft_config,
    max_seq_length=2048,
    dataset_text_field="messages"  # ChatML formatı için
)

# 6. Eğitimi başlat
trainer.train()

# 7. Modeli kaydet
trainer.save_model("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")

print("✅ Model eğitimi tamamlandı!")
'''

# Kodu dosyaya kaydet
with open(output_dir / 'ornek_finetuning.py', 'w', encoding='utf-8') as f:
    f.write(finetuning_kod)

print("Fine-tuning Örnek Kodu:")
print("=" * 80)
print(finetuning_kod)
print("=" * 80)
print(f"\n✅ Kod kaydedildi: {output_dir / 'ornek_finetuning.py'}")

## 8. Dosyaları İndirme

Oluşturduğunuz veri setlerini bilgisayarınıza indirin.

In [ ]:
# Google Drive'a kaydet (opsiyonel)
from google.colab import drive

try:
    drive.mount('/content/drive')
    
    # Veri setlerini Drive'a kopyala
    !mkdir -p '/content/drive/MyDrive/DeepFabric_Datasets'
    !cp -r sentetik_veriler/* '/content/drive/MyDrive/DeepFabric_Datasets/'
    
    print("✅ Veri setleri Google Drive'a kaydedildi!")
    print("   Konum: MyDrive/DeepFabric_Datasets/")
except Exception as e:
    print(f"⚠️  Drive bağlantısı kurulamadı: {e}")

In [ ]:
# Veya ZIP olarak indir
from google.colab import files
import shutil

# ZIP oluştur
shutil.make_archive('sentetik_veri_setleri', 'zip', 'sentetik_veriler')

print("✅ ZIP dosyası oluşturuldu!")
print("\nİndirmek için aşağıdaki satırın yorum işaretini kaldırın:")
# files.download('sentetik_veri_setleri.zip')

## 9. Özet ve Sonraki Adımlar

### 📋 Bu Notebook'ta Neler Yaptık?

✅ DeepFabric'i kurduk ve OpenRouter API ile yapılandırdık  
✅ GPT-OSS-20B modelini kullanarak sentetik veri ürettik  
✅ 5 farklı türde veri seti oluşturduk:
   - Temel veri seti
   - Python soru-cevap
   - Müşteri konuşmaları
   - Kod örnekleri
   - Tool calling
✅ Veri setlerini analiz ettik ve görselleştirdik  
✅ Fine-tuning için hazır hale getirdik  

### 🚀 Sonraki Adımlar

1. **Daha Fazla Veri Üretin**:
   - `num_steps` değerini artırın (50-100)
   - Farklı konular deneyin
   - Temperature ayarlarını optimize edin

2. **Veri Kalitesini Artırın**:
   - Prompt'ları daha spesifik hale getirin
   - Üretilen örnekleri manuel olarak inceleyin
   - Düşük kaliteli örnekleri filtreleyin

3. **Model Eğitimi**:
   - Ürettiğiniz veri ile küçük bir model fine-tune edin
   - LoRA/QLoRA kullanarak verimli eğitim yapın
   - Farklı hyperparameter'lar deneyin

4. **Değerlendirme**:
   - Fine-tune edilmiş modeli test edin
   - Benchmark sonuçlarını karşılaştırın
   - Kullanıcı geri bildirimi toplayın

### 📚 Faydalı Kaynaklar

- **DeepFabric**: https://github.com/lukehinds/deepfabric
- **OpenRouter**: https://openrouter.ai/
- **HuggingFace TRL**: https://huggingface.co/docs/trl/
- **PEFT/LoRA**: https://huggingface.co/docs/peft/

---

**İyi çalışmalar! 🎉**

## 10. Ekstra: Parametreleri Anlama ve Optimizasyon

### Konu Ağacı Parametreleri

- **`depth`**: Konu ağacının derinliği (kaç seviye alt konu)
  - `depth=1`: Sadece ana konu
  - `depth=2`: Ana konu + 1 seviye alt konular
  - `depth=3`: Ana konu + 2 seviye alt konular

- **`degree`**: Her konudan kaç alt konu türetilir
  - `degree=2`: Her konudan 2 alt konu
  - `degree=3`: Her konudan 3 alt konu
  - Toplam konu: yaklaşık `degree^depth`

### Veri Üretim Parametreleri

- **`num_steps`**: Kaç örnek üretilecek
- **`batch_size`**: Paralel üretim (API limitine dikkat)
- **`temperature`**: Yaratıcılık seviyesi
  - 0.0-0.3: Deterministik, tutarlı
  - 0.4-0.7: Dengeli
  - 0.8-1.0: Yaratıcı, çeşitli

### Optimizasyon İpuçları

1. **Maliyet Yönetimi**: OpenRouter ücretlidir
   - Önce `num_steps=5` ile test edin
   - Kaliteyi kontrol edin
   - Sonra artırın

2. **Hız Optimizasyonu**:
   - `batch_size=2-4` kullanın
   - Çok yüksek değerler API limitlerine takılabilir

3. **Kalite İyileştirme**:
   - Prompt'ları örneklerle zenginleştirin
   - Temperature'ı göreve göre ayarlayın
   - Üretilen veriyi düzenli inceleyin